# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

In [4]:
#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight;")

while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print("E")
        break

    last_height = new_height
    print(".", end="")

.........E


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 武漢防疫暑假延後放　長榮/立榮航空：旅客改、退票免收手續費

▲長榮/立榮航空今(3)日宣布，提供暑假期間已購票之出國旅客改、退票免收手續費服務。（圖／長榮航空提供）
記者林淑慧／台北報導
長榮/立榮航空今(3)日宣布，因應教育部防疫措施之寒假假期延後結束及暑假假期延後開始時間，將提供暑假期間已購票之出國旅客改、退票免收手續費服務，概述說明如下，詳細相關辦法及適用條件已公布於長榮航空官網www.evaair.com，歡迎民眾前往查詢。
武漢肺炎疫情持續延燒，除大陸武漢封城外，溫州當地民眾也被禁止來台，全球各地民眾減少亞洲行程，航空與觀光產業首當其衝，長榮航、華航今天股價大跌，盤中創下2009年下半年以來新低。
凡持有長榮/立榮航空兩岸或國際線6月30日至7月15日(含)由台灣出發之確認機位，並已於2月3日(含)前開立機票者，即日起至2月10日(含)，更改其他出發日期且與原開立機票相同之行程及訂位艙等，或是辦理退票，均免收一次手續費，使用哩程兌換之機票或是升等之機位亦比照辦理，但免費票或是折扣機票則不適用。
旅客透過長榮航空網站購買之機票，可直接於官網提出改、退票申請或請洽長榮航空訂位票務中心，經由旅行社購買之個人或是團體機票，則需由原開票旅行社協助辦理。
長榮/立榮航空持續密切注意疫情發展及市場需求，2月17日起往返兩岸及港、澳部分航班仍會依實際狀況調整，相關航班資訊將陸續於官網公告，最新航班異動資訊，敬請旅客於出發前，至長榮航空官網 www.evaair.com、EVA Mobile APP查詢。
----------------------------------------------------------------------

[1] 謝金河：台股通過初步的壓力測試　股市投機的過猶不及！

▲財信傳媒董事長謝金河。（圖／記者李瑞瑾攝）
記者謝仁傑／綜合報導
新型冠狀病毒（武漢肺炎）的疫情持續擴大，影響全球股市，財信傳媒董事長謝金河今（3）日在臉書連發兩文指出，中國A股恢復交易，對資本市場來說是好事，因為醜媳婦總要見公婆，今天收盤跌7.72％，算是尾盤有接手；台股已經跌了近千點，下檔抵抗力已經逐漸增強，「

▲VHQ-KY威馳克媒體公布北京等地製作團隊營運實景。（圖／VHQ-KY提供）
記者林淑慧／台北報導
武漢肺炎疫情持續延燒，中國大陸影視相關單位宣佈，暫停所有影視劇拍攝工作，電影院線也暫停開放；對此，影視特效製作公司VHQ-KY表示，這是防疫優先下不得已的決定，並坦言今年首季業績恐較去年下滑，也將下修全年業績目標。
中國國家廣播電視總局主管的中國廣播電視社會組織聯合會電視製片委員會、演員委員會於本月1日發布通知，在武漢肺帶疫情防控期間，要求所有影視製片公司、影視劇組及影視演員暫停影視劇拍攝工作。
針對武漢肺炎對中國影視行業及公司影響，影視特效製作公司VHQ-KY表示，後期製作市場相當龐大，VHQ-KY占市場份額僅2%左右，疫情對整體行業的衝擊力對於公司業績影響相對有限，而大陸影視劇市場依舊龐大且需求持續增長，相信在疫情結束之後將逐步恢復正常。
VHQ-KY表示，在疫情衝擊下，中國大陸影視行業也在進行改變和調整，以徐崢先生的作品《囧媽》為例，首次與線上平台合作進行網路播出，雖然是個案並且引起一些爭議，但畢竟是一種新的嘗試也得到了一定效果。另外，橫店影視城對所有暫定拍攝的劇組減免一切費用，並補貼群眾演員，顯示影視產業界團結一心共抗疫情。
展望後市，VHQ-KY表示，這次不可抗力的突發事件，勢必會給公司帶來不小的影響，尤其今年第一季與去年同期相比勢將下滑，全年業績目標預估也會調整下修。不過公司強調，疫情一旦結束，北京公司所有同事會加班加點，用一到二個財季把失去的時間全力彌補回來。
----------------------------------------------------------------------

[12] 跟隨蘋果腳步　三星宣布暫閉大陸的上海外灘旗艦體驗店

▲三星宣佈位於中國大陸上海的外灘旗艦體驗店將暫停營業。（圖／達志影像／美聯社）
記者陳心怡／外電報導
武漢肺炎疫情肆虐，許多店家都關閉營業，而基於公共衛生與預防的考慮，三星日前宣佈設於中國大陸上海外灘的旗艦體驗店，將暫停營業至2月9日，並表示「將繼續密切關注局勢，並希望盡快重新開放體驗店」。
三星也提醒，疫情期間，少出門，盡量避免去人多的地方是目前防範新型冠狀病毒最好的方式。
蘋果日前也發佈聲明「從即日起至2月9日，出於極其慎重考慮並根據專家的最新建議，我們將關閉在中國大陸的所有辦公室、零售

▲防疫照顧假事業單位願意給薪？人力銀行僅3成8願意。（圖／記者李毓康攝）
記者余弦妙／台北報導
「防疫視同作戰！」勞動部指出，因應延後兩週開學，若勞工朋友有照顧12歲以下學童需求，可以請「防疫照顧假」，並要求雇主不能不准假，目前最多為兩個星期，也不得影響出缺勤、考績，只是是否給薪，「全依公司規定」。yes123求職網發言人楊宗斌預估，能得到有薪防疫照顧假，恐怕不太容易。
因為根據yes123求職網，去年四月就針對「家庭照顧假」的調查，若以「資方」態度來看，合計有38.5%的企業「願意支付薪水」，其中32.2%「願意支付半薪」；6.3%「願意支付全薪」。代表剩下61.5%的企業認為，「維持無薪即可」。
楊宗斌表示，想請「防疫照顧假」的勞工，若擔心薪資減少問題，可能改採「全薪」特休假或「半薪」病假因應；不然就是衡量送安親或托育機構的花費成本。
對企業而言，除了擔心職場防疫，與照顧假帶來的人力吃緊問題，像是外銷導向業者，對岸產線何時復工，以及台幹上工狀況，短期將影響到首季業績；對內需導向業者來說，則開始擔心民眾外出消費情況，不過反倒使「外送員」職缺，出現增加的可能性。
但對於這次勞動部的積極反應作法，孫友聯則是表達肯定，由於這次疫情非同小可，的確有擴大可能，個人會選擇相信中央流行疫情指揮中心的判斷。
----------------------------------------------------------------------

[24] 武漢肺炎疫情延燒　航運界聯合春節團拜活動取消

▲武漢肺炎疫情持續延燒，原訂明（4）日上午舉行的航運界聯合新春團拜活動取消舉辦。（圖／資料照）
記者林淑慧／台北報導
武漢肺炎疫情持續延燒，不少公司、政府單位相繼取消新春團拜，一年一度的航運界聯合新春團拜活動，經籌備委員會評估疫情狀況，為求謹慎，將取消原訂明（4）日上午舉行的新春團拜活動。
航運界春節團拜包括輪船商業同業公會全聯會、船務代理商業同業公會全聯會、中華海員服務中心、財團法人中國驗船中心、船長公會、中華海員總工會等24個航運界團體共同舉行。
中華民國航運界春節團拜籌備會表示，武漢肺炎疫情持續延燒，各機關陸續暫停或延期大型活動，面對肺炎疫情，政府已有完整防疫措施，呼籲民眾不必過分恐慌，但也不應鬆懈，相信政府與人民可以做好全方位的防護。
另外，因中國大陸延後開工造成供應鏈

 ▲張世傑炒作正峰股票，最高法院判刑定讞。（圖／《ETtoday新聞雲》資料照）
記者吳銘峯／台北報導
股市炒手「古董張」張世傑，在2001、2002年間，夥同上市公司正峰新能源董事長鍾明甫夫婦，共同炒作正峰股票，一度將每股18元炒高到27元，不法獲利達4322萬餘元。檢方提起公訴後，鍾明甫夫婦陸續遭判刑定讞。至於張世傑涉案部分，最高法院3日依照違反《證券交易法》「操縱證券交易價格罪」，判刑6月定讞。
張世傑因炒股而官司纏身，他因炒作唐鋒股票遭最高法院判刑8年6月定讞，2016年3月間企圖從馬祖出海、偷渡中國福建，遭海巡官兵攔截查獲，入監執行。他在監期間仍陸續遭判刑，去年高等法院針對他與「世界陳」稱號的上櫃天剛資訊前董事長陳和宗，共同炒作天剛公司股票，再將張世傑判刑4年，目前上訴最高法院審理中。
至於最高法院這次判刑定讞部分，是因2001年11月間，時任上市正峰工公司（後改名為正峰新能源公司）董事長鍾明甫，為吸引外資及法人買進正峰工股票，而與證券營業員張嘉元共謀炒作；並由鍾妻、鍾黃玉華及張嘉元各提供人頭戶相對買賣方式，製造交易熱絡。張嘉元還引進張世傑與麗天投顧蔡漢凱等炒手一起炒作，半年時間從每股18元炒至每股24.9元。
不過正峰公司股價於2002年1月中旬，仍回跌至19元左右。張嘉元為求出脫持股，與張世傑、蔡漢凱合謀炒股，協議在股票拉高後賣得之價差，一半歸由張世傑，另一半歸由張嘉元與蔡漢凱均分，在同年3月下旬將每股炒到27元以上，張世傑等人出脫持股，獲利達4322萬多元。台中地檢署偵查終結後，2011年間對張世傑、鍾明甫夫婦等人提起公訴。
鍾明甫夫婦最後遭判刑4年、1年10月確定，張嘉元於2015年間因病過世，蔡漢凱則逃亡遭通緝至今；至於張世傑也因官司纏身，企圖潛逃遭發布通緝，直到2016年才被逮捕歸案。張世傑一審遭判刑1年8月，二審因認定張涉案部分與佳和炒股案有同一性，做出不受理判決，最高法院撤銷發回後，台中高分院更一審改判1年、減為6月。全案再上訴最高法院，3日駁回上訴定讞。
----------------------------------------------------------------------

[36] 雲端最有錢／武漢肺炎重創中國經濟　台灣第二慘？能危機入市嗎？

新聞節目中心／綜合報導
2020真是不平靜的一年，中國武漢肺炎病毒來勢

▲台股示意圖。（圖／記者湯興漢攝）
台股今（3）日早盤一度跌破300多點，引發投資人關注，外界關心國安基金的動向，國安基金官員表示，早盤跌勢稍有收斂，持續觀察當中，但是台股基本面真的良好，請投資人不要過度恐慌。
台股自開紅盤以來，因為武漢疫情擴散，台股表現大幅波動，雖上周跌勢一度暫歇，但隨疫情升溫，加上陸股於今日開市，讓投資人不安情緒漸漲，台股今開盤亦下跌129點、以11,365點開出，盤中甚至一度大跌了300多點，顯然投資人的恐慌氣氛仍在。
國安基金官員表示，中國大陸股市今日開市，持續觀察陸股的走向；但其他的國際股市，像是日韓股表現穩定、美股周五（1月31日）等卻有下滑。
官員表示，目前台股只是短暫的波動，目前還在觀察，如果有失序的情況，就會報請行政院國安基金委員會召開委員會議。但是官員強調，台股主要是反應基本面，台灣的基本面真的很不錯，請投資人不要過度恐慌。
----------------------------------------------------------------------

[48] 新台幣開盤重貶1角　以30.35元兌1美元開出

▲ 新台幣兌美元今天開盤貶值1角。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（3）日開盤貶值1角、以30.35元兌1美元開出。
新台幣兌美元上週五（31日）收在30.25兌1美元；今天台股開盤跌逾100點，並持續下挫，新台幣匯率開盤也重貶1角，以30.35元兌1美元開出。
新台幣開盤後貶勢收斂，截至上午9點11分，新台幣來到30.31元兌1美元，貶值6分。
----------------------------------------------------------------------

[49] 台股早盤大跌逾300點！陸股開市引發不安　台股跌破11200點

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
武漢疫情持續衝擊全球金融市場，雖上周跌勢一度暫歇，但隨疫情升溫，加上陸股於今（3）日開市，讓投資人不安情緒漸漲，台股今開盤亦下跌129點、以11,365點開出。
截至上午9點04分，加權指數更是下跌超過300點，來到11,193點。
電子三雄中，台積電（2330）以315元開出、下跌3.53%；鴻海（2317）開在79.5元、下跌4.44%；股王大立光（300

▲勞動部長許銘春（左二）指出，因應開學延後，創設「防疫照顧假」，父母其中一人最多可以請到14天假。（圖／記者周宸亘攝）
記者邱倢芯／台北報導
武漢肺炎疫情持續延燒，教育部決議高級中等以下學校開學日延後2週，不過這項決議勢必將影響家長工作；因此勞動部今（2）日宣布，創設「防疫照顧假」，家中如有12歲以下兒童，父母其中一人最多可以請到14天假，除了要求雇主不得拒絕外，還要求不得扣全勤。
只不過勞工請防疫照顧假雇主需不需給薪？勞動部長許銘春並沒有正面回應，僅在記者會中強調，這是為了因應這次的武漢肺炎疫情所創設的假別，並非公假的一種；勞工提出「防疫照顧假」請假申請時，雇主不得拒絕，且不得視為缺勤而影響全勤獎金、考績，或為其他不利的處分。至於要不要給薪看各公司規定，企業則可以做研議。
----------------------------------------------------------------------

[62] 亞洲前3大塑橡膠專業展Taipei PLAS　9月在台登場　

▲2018年台北國際塑橡膠工業展展場實況。（圖／貿協提供）
記者余弦妙／台北報導
兩年一度的「台北國際塑橡膠工業展（Taipei PLAS）」將在今年9月9日至13日在南港展覽1館隆重登場。外貿協會表示，今年主打「工業4.0」、「智慧製造」及「循環經濟」3大主題，並新增產業應用面，整合包裝及印刷設備，展現臺灣塑橡膠產業供應鏈實力，另搭配採購洽談會、高峰論壇及技術研討會等豐富多元周邊活動，打造全方位展示與拓銷平台。
貿協說，因應全球原油價格上漲導致材料成本增加，且全球環保意識抬頭，如何提高塑膠廢棄物回收與再生率，將是塑橡膠產業當前最重要的課題，其中，塑橡膠產品開發過程導入綠色設計、節能製程、生質材料使用及回收等，更是業者共同的目標。
因此，Taipei PLAS今年順應趨勢，首次將「循環經濟」列為主題之一，鼓勵業者展出低汙染的設備及智慧機械，以新研發之製程技術，達到少廢料、低耗能、提高製造效率及再生原料品質為訴求，在塑膠製造及環境永續發展中取得平衡，創造嶄新商機。
貿協指出，台灣為全球前六大橡塑膠機械出口國，在2018年時，出口值達到11.7億美元， Taipei PLAS也穩居亞洲前3大、全台第一大的塑橡膠專業展。
2020年Taipei PLAS預計有550家國內外廠商參展，將使

▲台北民眾搶買口罩，超商外大排長龍。（圖／記者陳俊宏攝）
文／中央社記者廖禹揚台北1日電
經濟部規劃新建62條口罩產線，將在3日起的一個月內陸續上線，人力不足部分正洽請國防部支援；經濟部也呼籲民眾口罩夠用就好，不要囤積。
因應防疫口罩需求，經濟部規劃出資新建62條產線，官員對中央社記者表示，目前正與國防部接洽，調派人力支援生產人力，但尚未確認詳細方案，工業局也正同步調查廠商人力需求。
民眾搶購口罩、消毒酒精等防疫用品，雖實施每人3片口罩的購買限制，仍有許多民眾反映無法買到口罩。經濟部也透過臉書粉絲專頁貼文呼籲民眾不要囤積口罩。
▼現在口罩十分難買。（圖／記者周亭瑋攝）
經濟部表示，短期內口罩進口價錢提高，也不易取得，台灣必須強化國內供應能力，以滿足國人需求，行政院已拍板以最快速度新建62條口罩產線，包括10%兒童口罩模具，現已盤點完畢機器放置與生產空間，並委託國內口罩專業生產工廠協助製造。
貼文指出，3日起開始的一個月內，62條產線會陸續上線，產能可從目前的每日400萬片逐步提高至每日1000萬片，讓國內供應更穩定。
經濟部說，PP聚丙烯（不織布原料）、到口罩內層ES、中間層熔噴布、外層紡黏等原物料在國內都有生產，經濟部已掌握供應狀況。
延伸影音：
----------------------------------------------------------------------

[74] 保單借款「不超過保價金90%」！低利率貸款優先選擇　發揮資金最高效益

▲保單借款一定要注意償還時間。（圖／pixabay）
財經中心／台北報導
若臨時有資金需求或緊急用途，民眾拿不出現金應急，除了可向銀行申請貸款，也可考慮透過保單貸款，用購買的保險商品累積的保單價值準備金，作為資金周轉。不過，金管會提醒，一般貸款額度不會超過保單價值準備金的90%，一旦不幸發生保險事故，貸款本息還會從理賠保險金中扣除。
民眾購買的壽險類商品若具有累積保單價值準備金（保價金），就可向保險公司辦理保單貸款。保單貸款為保戶權益項目之一，保單所有人可直接將有累積保價金的保單作為擔保物，獲取貸款。
至於貸款額度，金管會指出，實務上一般貸款額度不會超過保單價值準備金的90%，因此保戶需要注意，若不幸發生保險事故，貸款本息將從理賠的保險金中扣除；若貸款本息超過保單價值準備金，則保單將停效，而在停效期

▲桃園市中路特區「京懋一號」建案推出「減房概念」，吸引中高消費族群享受寬裕空間居住環境。（圖／業者提供）
記者沈繼昌／桃園報導
位於桃園市中路特區的「京懋一號」是京懋建築團隊近年特別為中高消費族群精心設計的個案，其中最吸睛之處是「減房概念」，主打享受寬裕空間、開放式廚房設計與寬敞公設等特點提升居住品質，讓喜歡「房間不必多，空間不能少」的消費族群，可將居家、互動重心置於公共領域，推案之後果然讓中高消費族群趨之若鶩。
▲桃園市中路特區「京懋一號」建案的公共空間咖啡廳典雅時尚。（圖／業者提供）
「京懋一號」個案推出68坪2房產品，在桃園建案市場上尚屬特例，京懋建設執行長范秉豐指出，「減房概念」能透露出居住者的生活內涵，以中大坪數空間卻只設計2個房間，讓更多的生活空間放在客廳與餐廳，如書房、廚房區域並非像坊間推出＋1空間，以提高臺度方式區隔，開放形式融入在空間設計。
范秉豐解釋，近年餐廳在住家重要性逐漸提升，許多家庭聚會宴客改以選擇在餐廳而非客廳，充足中島空間與開放式廚房設計則是建築團隊規劃重點，親友來訪時，主人可一邊小露廚藝，一邊閒話家常，其中L型廚房空間，設計全套義大利歐式精品廚具，讓品味在開放空間自然流露。
▲桃園市中路特區「京懋一號」建案的公共空間擁有絕佳的庭園戶外景觀視覺。（圖／業者提供）
此外，主臥室透過減房概念也與一般傳統三至四房設計不同，床組與牆面之間可讓大型行李箱完整攤開，讓喜愛旅行居家者不必將行李箱提到客廳整理，「京懋一號」獨特減房概念讓居家空間更為多樣化，更增添主客互動性，再利用光線與空氣的流通，讓視覺空間更為一體。
▲桃園市中路特區「京懋一號」建案的大廳吊燈典雅大方。（圖／業者提供）
►點我超越韓國愉快體驗
----------------------------------------------------------------------

[88] 「非常不當！」張金鶚批台中降地價稅：只是讓大地主業者更容易養地獲利

記者董美琪／綜合報導
台中市決議109年公告地價調降20.06%，讓財政部氣得揚言要「拿出棍子」，表示可能減少對台中的統籌分配稅款，台中市府聲明強調這可以嚷超過9成納稅人受益，一切都是嚴謹調降「還稅於民」」。對此前台北市副市長暨政治大學教授張金鶚認為「台中市政府大幅調降公告地價作法，非常不當」。
張金鶚發文表示，台中市政府大幅

In [7]:
# 關閉瀏覽器
browser.quit();